# Streaming with Kafka and Spark

Here I try to implement a basic pipeline for the project conecting kafka with spark.

### Computer setting
I downloaded and located in my home the spark file **spark-3.1.2-bin-hadoop3.2** and also the kafka file **kafka_2.13-2.7.0**

## Get Kafka and Spark ready

### Standalone cluster deployment

We can now initialize all the required variables with `findspark.init()` by passing the path to the spark folder we downloaded previously.

In [1]:
import findspark
findspark.init('/usr/local/spark')

First we need to start the master, This will spin up the spark master with address spark://localhost:7077 and a cluster dashboark at localhost:8080.

We can now create a worker

In [2]:
%%script bash --no-raise-error

# start master 
$SPARK_HOME/sbin/start-master.sh --host localhost \
    --port 7077 --webui-port 8080
    
# start worker
$SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
    --cores 4 --memory 2g



starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark/logs/spark-root-org.apache.spark.deploy.worker.Worker-1-mapd-b-gr17-5.novalocal.out
starting org.apache.spark.deploy.master.Master, logging to /usr/local/spark/logs/spark-root-org.apache.spark.deploy.master.Master-1-mapd-b-gr17-5.novalocal.out


## Create the spark session

We can now create the spark session. With the following command we are asking to the master (and resource manager) to create an application with required resources and configurations. In this case we are using all the default options.

In [2]:
from pyspark.sql import SparkSession

KAFKA_BOOTSTRAP_SERVERS = ''


spark = SparkSession.builder \
    .master("spark://master:7077")\
    .appName("Spark Streaming")\
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")\
    .getOrCreate()


In [3]:
spark

## KAFKA

In [4]:
KAFKA_HOME = '/usr/local/kafka'
KAFKA_BOOTSTRAP_SERVERS = 'slave01:9092'

In [5]:
#By some reason I can't launch this from here using OS, so i open the terminals in the KAFKA_HOME folder
# and launch the zookeper and the kafka server comands manually


# Start Zookeeper
# bin/zookeeper-server-start.sh config/zookeeper.properties 
#os.system('{0}/bin/zookeeper-server-start.sh {0}/config/zookeeper.properties'.format(KAFKA_HOME)) 
    
# Start one Kafka Broker
#bin/kafka-server-start.sh config/server.properties
#os.system('{0}/bin/kafka-server-start.sh {0}/config/server.properties'.format(KAFKA_HOME)) 

### Create the topics for kafka

In [9]:
from kafka.admin import KafkaAdminClient, NewTopic
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    )

#Here we will inject the data
new_topic_a = NewTopic(name='Experiment_measurements', 
                       num_partitions=1, 
                       replication_factor=1)

#Here we inject the number of processed hits, post cleaning
new_topic_b = NewTopic(name='results', 
                       num_partitions=1, 
                       replication_factor=1)

kafka_admin.create_topics(new_topics=[new_topic_a,new_topic_b])


CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='Experiment_measurements', error_code=0, error_message=None), (topic='results', error_code=0, error_message=None)])

In [10]:
kafka_admin.list_topics()

['results', 'Experiment_measurements']

## KAFKA - SPARK INTEGRATION

### Read the data from the kafka topic (define the consumer)

In [11]:
inputDF = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
    .option('subscribe', 'Experiment_measurements')\
    .load()

In [12]:
#inputDF.printSchema()

In [13]:
from pyspark.sql.functions import from_json, col, when
from pyspark.sql.types import StructField, StructType, DoubleType, IntegerType

## the schema of the json data format used to create the messages
schema = StructType(
        [
                StructField("HEAD",        IntegerType()),
                StructField("FPGA",        IntegerType()),
                StructField("TDC_CHANNEL", IntegerType()),
                StructField("ORBIT_CNT",   DoubleType()),
                StructField("BX_COUNTER",  IntegerType()),
                StructField("TDC_MEAS",    DoubleType())
        ]  
)

## a new DF can be created from the previous by using the pyspark.sql functions
jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

In [14]:
jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: integer (nullable = true)
 |    |-- FPGA: integer (nullable = true)
 |    |-- TDC_CHANNEL: integer (nullable = true)
 |    |-- ORBIT_CNT: double (nullable = true)
 |    |-- BX_COUNTER: integer (nullable = true)
 |    |-- TDC_MEAS: double (nullable = true)



In [15]:
#jsonDF.writeStream\
#   .outputMode("append")\
#   .format("console")\
#   .start()\
#   .awaitTermination()

In [16]:
flatDF = jsonDF.selectExpr("value.HEAD", 
                           "value.FPGA", 
                           "value.TDC_CHANNEL",
                           "value.ORBIT_CNT",
                           "value.BX_COUNTER",
                           "value.TDC_MEAS")

In [17]:
flatDF.printSchema()

root
 |-- HEAD: integer (nullable = true)
 |-- FPGA: integer (nullable = true)
 |-- TDC_CHANNEL: integer (nullable = true)
 |-- ORBIT_CNT: double (nullable = true)
 |-- BX_COUNTER: integer (nullable = true)
 |-- TDC_MEAS: double (nullable = true)



In [18]:
#flatDF.writeStream\
#   .outputMode("append")\
#   .format("console")\
#   .start()\
#   .awaitTermination()

### SPARK processing

In [19]:
import json
import numpy as np
import time

#Keep the events where "HEAD"=2
cleanDF = flatDF.where(col('HEAD')==2)  

In [20]:
def computations(DF, epoch):
    #This function perform the whole operations on the received batch,
    #

    #As the 4 calculations that we have to perform are done foe each chamber we set 4 dataframes
    chamber_1 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_2 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))
    chamber_3 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_4 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))

    #Now we can count the number of events in each chamber
    n_c1 = chamber_1.count()
    n_c2 = chamber_2.count()
    n_c3 = chamber_3.count()
    n_c4 = chamber_4.count()

    #Total number of events
    n = n_c1 + n_c2 + n_c3 + n_c4


    #Histograms    
    h_c1 = chamber_1.groupBy('TDC_CHANNEL').count().collect()
    h_c2 = chamber_2.groupBy('TDC_CHANNEL').count().collect()
    h_c3 = chamber_3.groupBy('TDC_CHANNEL').count().collect()
    h_c4 = chamber_4.groupBy('TDC_CHANNEL').count().collect()

    h_active_1 = chamber_1.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_2 = chamber_2.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_3 = chamber_3.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_4 = chamber_4.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    
    
    #Organise the results to send them to one topic as a dictionary
    results = {'Total_events': n,
              'Events_per_chamber': [n_c1,n_c2,n_c3,n_c4],
              'Histogram_1': [h_c1, h_c2, h_c3, h_c4],
              'Histogram_2': [h_active_1,h_active_2,h_active_3,h_active_4]}
    
    #publish the results in the "results" topic for further usage
    producer.send(topic='results', value=json.dumps(results).encode('utf-8'))
    producer.flush()

In [21]:
def computations_2(DF, epoch):
    start=time.time()
    #This function perform the whole operations on the received batch,
    #

    #As the 4 calculations that we have to perform are done foe each chamber we set 4 dataframes
    chamber_1 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_2 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))
    chamber_3 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_4 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))

    #Initialize results dictionary
    results = {}
    results["Total Count"] = {}
    chamber_name = ["Chamber_1", "Chamber_2", "Chamber_3", "Chamber_4"]
    for chamber in chamber_name:
        results[chamber] = {}
        results[chamber]["Count"] = {}
        
    #Now we can count the number of events in each chamber
    results["Chamber_1"]["Count"] = chamber_1.count()
    results["Chamber_2"]["Count"] = chamber_2.count()
    results["Chamber_3"]["Count"] = chamber_3.count()
    results["Chamber_4"]["Count"] = chamber_4.count()

    #Total number of events
    results["Total Count"] = results["Chamber_1"]["Count"] + results["Chamber_2"]["Count"] + \
                             results["Chamber_3"]["Count"] + results["Chamber_4"]["Count"]
    

    # Compute histograms for each chamber   
    i=0    
    for chamber in [chamber_1, chamber_2, chamber_3, chamber_4]:
        #Initialize dictionary partitions to save the results
        for hist in ["Hist_1","Hist_2"]:
            results[chamber_name[i]][hist] = {}
            results[chamber_name[i]][hist]["Bins"] = {}
            results[chamber_name[i]][hist]["Counts"] = {}
        
        if(chamber.count()!=0): 
            #Histogram 1
            bins, counts = (
                chamber.select("TDC_CHANNEL")
                .rdd.map(lambda x: x.TDC_CHANNEL)
                .histogram(list(np.arange(0,170,5)))
            )
            
            results[chamber_name[i]]["Hist_1"]["Bins"] = bins
            results[chamber_name[i]]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            bins, counts = (
            chamber.groupBy("TDC_CHANNEL","ORBIT_CNT")
            .count()
            .select("ORBIT_CNT")
            .rdd.map(lambda x: x.ORBIT_CNT)
            .histogram(list(np.arange(6.e5,1.e7,0.5e6)))
            )
            
            results[chamber_name[i]]["Hist_2"]["Bins"] = bins
            results[chamber_name[i]]["Hist_2"]["Counts"] = counts            
            
        else:
            #Histogram 1
            results[chamber_name[i]]["Hist_1"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[chamber_name[i]]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            results[chamber_name[i]]["Hist_2"]["Bins"] = list(np.arange(6.e5,1.e7,0.5e6))
            counts = list(np.arange(6.e5,1.e7,0.5e6)* 0) 
            results[chamber_name[i]]["Hist_2"]["Counts"] = counts
        i +=1
    
    end =time.time()
    print("Time =",end-start)
    producer.send(topic="results", value= str(results).encode('utf-8'))
    producer.flush()

In [25]:
def computations_3(DF, epoch):
    start=time.time()
    #This function perform the whole operations on the received batch,
    
    #Add a column with the chamber number
    DF_new = DF.withColumn('chamber',when((col("FPGA") == 0) & (col("TDC_CHANNEL")<=63),1).
                                 when((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64),2).
                                 when((col("FPGA") == 1) & (col("TDC_CHANNEL")<=63),3).
                                 when((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64),4)).\
                                 select([ col('TDC_CHANNEL'), col('ORBIT_CNT'),
                                    col('BX_COUNTER'),col('TDC_MEAS'),
                                    col('chamber')])
    #DF_new.persist()
    #DF_new.show()
    #Initialize results dictionary
    results = {}
    results["Total Count"] = {}
    results["Index"] = time.time()
    chamber_name = ["Chamber_1", "Chamber_2", "Chamber_3", "Chamber_4"]
    for chamber in chamber_name:
        results[chamber] = {}
        results[chamber]["Count"] = {}
        for hist in ["Hist_1","Hist_2"]:
            results[chamber][hist] = {}
            results[chamber][hist]["Bins"] = {}
            results[chamber][hist]["Counts"] = {}
        
    # Compute histograms for each chamber   
    for i in [1,2,3,4]:
        #Now we can count the number of events in each chamber
        chamber = DF_new.filter(col("chamber") == i).persist()
        results[f"Chamber_{i}"]["Count"] = chamber.count()
        
        if(results[f"Chamber_{i}"]["Count"]!=0):
            
            #Histogram 1
            bins, counts = (
            chamber.select("TDC_CHANNEL")
                 .rdd.map(lambda x: x.TDC_CHANNEL)
                 .histogram(list(np.arange(0,170,5)))
            )
            
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = bins
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            bins, counts = (
            chamber.groupBy("TDC_CHANNEL","ORBIT_CNT")
            .count()
            .select("ORBIT_CNT")
            .rdd.map(lambda x: x.ORBIT_CNT)
            .histogram(list(np.arange(6.e5,1.e7,0.5e6)))
            )
            
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = bins
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts            
                
        else:
            #Histogram 1
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts
            
             #Histogram 2
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = list(np.arange(6.e5,1.e7,0.5e6))
            counts = list(np.arange(6.e5,1.e7,0.5e6)* 0) 
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts
        chamber.unpersist()
        
    results["Total Count"] = results["Chamber_1"]["Count"] + results["Chamber_2"]["Count"] + \
                             results["Chamber_3"]["Count"] + results["Chamber_4"]["Count"]
    end =time.time()
    print("Time =",end-start)
       
    
    producer.send(topic="results", value= str(results).encode('utf-8'))
    producer.flush()

Time = 12.103435754776001


In [23]:
from kafka import KafkaProducer

#Send the results to the kafka topic
#Initialize the producer
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)


In [26]:
#Trigger the processing
cleanDF.writeStream\
    .foreachBatch(computations_3)\
    .trigger(processingTime='6 second')\
    .start()\
    .awaitTermination()

Time = 2.7600951194763184
Time = 12.605599880218506
Time = 10.716827154159546
Time = 12.415242433547974
Time = 12.52517819404602
Time = 14.638943195343018
Time = 15.040907144546509
Time = 12.215430736541748
Time = 13.009719133377075
Time = 12.16805100440979
Time = 12.722002744674683
Time = 12.323020935058594
Time = 12.71977972984314
Time = 11.915708541870117
Time = 12.621934175491333
Time = 14.815287590026855
Time = 12.395026683807373
Time = 12.24831771850586
Time = 12.11865520477295
Time = 14.695249557495117
Time = 14.9103102684021
Time = 12.375563621520996
Time = 12.625203847885132
Time = 12.447861671447754
Time = 12.698059797286987
Time = 11.619881868362427
Time = 12.63197946548462
Time = 12.21250319480896
Time = 12.536131620407104
Time = 12.289501190185547
Time = 12.498435974121094
Time = 12.261037588119507
Time = 12.562913417816162
Time = 12.243067026138306
Time = 15.31281042098999
Time = 14.91160535812378
Time = 15.279619216918945
Time = 14.741284370422363
Time = 12.7635927200317

KeyboardInterrupt: 

In [27]:
spark.stop()

If you also want to delete any data of your local Kafka environment including any events you have created along the way, run the command:

`` $ rm -rf /tmp/kafka-logs /tmp/zookeeper `` 